# Capture odds - election winner at sportsbet

Note: this notebook is for ease of testing. Convert to a python file and move to the automated directory.

To do this ...
```
jupyter nbconvert --to python _Capture-odds-election-winner-at-sportsbet.ipynb

chmod 700 _Capture-odds-election-winner-at-sportsbet.py

mv _Capture-odds-election-winner-at-sportsbet.py ../automated
```

Shell file called by cron:
```
#!/bin/zsh
#crontab: 57 11 * * * /Users/bryanpalmer/Australian-Federal-Election-2025/automated/election-winner.sh

# set-up parameters
home=/Users/bryanpalmer
working=Australian-Federal-Election-2025/automated
runrun=_Capture-odds-election-winner-at-sportsbet.py
mmenv=p311

# move to the home directory
cd $home

# activate the micromamba environment
micromamba activate $mmenv

# move to the working directory
cd $working

#initiate the data capture
$home/micromamba/envs/$mmenv/bin/python ./$runrun >>log.log 2>> err.log
```

## Python setup

In [1]:
# data science imports
# System imports
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# web scraping imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

## Set-up web-driver options

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
options.add_argument("--headless")

## Extract website text using Selenium

In [3]:
# get the web page text
driver = driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install()), options=options
)
driver.implicitly_wait(60)
url = (
    "https://www.sportsbet.com.au/betting/politics/australian-federal-politics/"
    "48th-parliament-of-australia-6560505"
)
driver.get(url)
soup = BeautifulSoup(driver.page_source, "lxml")
driver.close()

In [4]:
# print(soup.prettify())

## Extract data of interest

In [5]:
# Step one - get to the right bit of the page ...
# <div class="background_fja218n" data-automation-id="content-background">
div_name = "content-background"  # this looks fragile
div = soup.find_all("div", {"data-automation-id": div_name})[0]
# print(div.prettify())

In [6]:
# step 2
# <div class="outcomeCardItems_f4kk892">
odds = div.find_all("div", {"class": "outcomeCardItems_f4kk892"})  # fragile
len(odds)  # should be two: for a names row and a prices row

2

In [7]:
found = {}
for k, v in zip(odds[0].contents, odds[1].contents):
    found[k.text] = v.text
found  # should be a dictionary of parties and prices/odds

{'Labor': '1.60', 'Coalition': '2.40', 'Any Other Party': '101.00'}

## Append this data to a CSV file

In [8]:
# long format
df = pd.DataFrame([found.keys(), found.values()], index=["variable", "value"]).T
df.index = np.repeat(datetime.datetime.now(), len(df))
df.index.name = "datetime"

In [9]:
df

,variable,value
datetime,,
2023-10-29 09:00:42.183511,Labor,1.60
2023-10-29 09:00:42.183511,Coalition,2.40
2023-10-29 09:00:42.183511,Any Other Party,101.00


In [10]:
# save to file
DIR = "../betting-data"
Path(DIR).mkdir(parents=True, exist_ok=True)
FILE = f"{DIR}/sportsbet-2025-election-winner.csv"
df.to_csv(FILE, mode="a", index=True, header=False)

## Done